# openFDA APIs

We are fetching data from the following openFDA APIs:

[Drug Product Labeling API](https://open.fda.gov/apis/drug/label/)

* **Overview:** The openFDA drug product labeling API returns data that has been collected from the Structured Product Labeling (SPL), a database that contains product labeling information submitted by manufactures and distributors to the FDA.

[Drug Adverse Events API](https://open.fda.gov/apis/drug/event/)

* **Overview:** The openFDA drug adverse event API returns data that has been collected from the FDA Adverse Event Reporting System (FAERS), a database that contains information on adverse event and medication error reports submitted to FDA.

In [ ]:
import pandas as pd
import requests
import time
import random

# Helper Function

**Retrieves results from the API endpoint by repeatedly requesting data in batches of 1000.**

In [2]:
def fetch_results(url, limit=1000, sleep_range=(0.2, 0.4)):
    all_results = []
    skip = 0

    while True:
        params = {
            "limit": limit,
            "skip": skip
        }

        data = requests.get(url, params=params).json()

        if "results" not in data:
            break

        all_results.extend(data["results"])
        skip += limit

        time.sleep(random.uniform(*sleep_range))

    return all_results

# Drug Product Labeling

**Retrieve FDA drug product labeling records.**

In [3]:
dpl_url = "https://api.fda.gov/drug/label.json"

all_results = fetch_results(dpl_url)

dpl_df = pd.json_normalize(all_results, errors="ignore")

**Keep and rename only crucial attributes.**

In [4]:
keep_columns = [
    "active_ingredient",
    "openfda.application_number"
  ]

clean_dpl_df = dpl_df[keep_columns]

clean_dpl_df = clean_dpl_df.rename(columns = {
    "active_ingredient": "active_ingredient",
    "openfda.application_number": "application_number"
  })

**Perform necessary cleaning.**

In [5]:
clean_dpl_df = clean_dpl_df.explode("application_number")

clean_dpl_df = clean_dpl_df.dropna(subset = ["application_number"])

clean_dpl_df["application_number"] = clean_dpl_df["application_number"].astype(str).str.lower()

clean_dpl_df = clean_dpl_df.dropna(subset = ["active_ingredient"])

clean_dpl_df['active_ingredient'] = (
    clean_dpl_df['active_ingredient']
        .astype(str)
        .str.replace(r'active\s*i+ngredient[s]?', '', case=False, regex=True)
        .str.replace(r'\bpurpose\b', '', case=False, regex=True)
        .str.replace(r'\bdrug facts\b', '', case=False, regex=True)
        .str.replace(r'anti[- ]?inflammatory\s+drug', '', case=False, regex=True)
        .str.replace(r'\bnonsteroidal\b', '', case=False, regex=True)
        .str.replace(r'\b(uv filter|sun ?screen|sun protection|spf)\b', '', case=False, regex=True)
        .str.replace(r'\d+(\.\d+)?\s*(mg|g|mcg|µg|kg|ml|mL|L)\b', '', regex=True)
        .str.replace(r'\b(usp|ip|bp)\b', '', case=False, regex=True)
        .str.replace(r'\s{2,}', ', ', regex=True)
        .str.replace(r'\.{2,}', ' ', regex=True)
        .str.replace(r'\([^)]*\)', '', regex=True)
        .str.replace(r'\\[A-Za-z0-9]+', '', regex=True)
        .str.replace(r'[^\w\s,]', '', regex=True)
        .str.replace(r'\d+', '', regex=True)
        .str.replace(r'\s+', ' ', regex=True)
        .str.strip()
)

clean_dpl_df['active_ingredient'] = clean_dpl_df['active_ingredient'].str.split(',')

clean_dpl_df = clean_dpl_df.explode('active_ingredient')

clean_dpl_df['active_ingredient'] = clean_dpl_df['active_ingredient'].str.strip().replace('', pd.NA)

clean_dpl_df['active_ingredient'] = clean_dpl_df['active_ingredient'].str.lower()

clean_dpl_df = clean_dpl_df.dropna(subset=['active_ingredient'])

clean_dpl_df.to_csv("clean_dpl.csv", index = False)

In [6]:
df = pd.read_csv("clean_dpl.csv")
df

,active_ingredient,application_number
0,povidoneiodine,m003
1,benzalkonium chloride,505g(a)(3)
2,salicylic acid,m006
3,hydrocortisone,m017
4,acetaminophen,m013
...,...,...
7737,acetaminophen,m012
7738,dextromethorphan hbr,m012
7739,guaifenesin,m012
7740,phenylephrine hcl,m012


# Drug Adverse Events

**Retrieve FDA drug adverse event records in batches of 1000.**

In [7]:
dae_url = "https://api.fda.gov/drug/event.json"

all_results = fetch_results(dae_url)

**Keep only crucial attributes; Flatten the patient drug information.**

In [8]:
expanded_dae_df = pd.json_normalize(
    all_results,
    record_path = ["patient", "drug"],
    meta = [
        "serious",
        "seriousnessdeath",
        "seriousnessdisabling",
        "seriousnessother",
        "seriousnesshospitalization",
        "seriousnesslifethreatening",
        "seriousnesscongenitalanomali"
    ],
    errors = "ignore"
)

**Keep and rename only crucial attributes; Flatten application numbers**

In [9]:
keep_columns = [
    "openfda.application_number",
    "serious",
    "seriousnessdeath",
    "seriousnessdisabling",
    "seriousnessother",
    "seriousnesshospitalization",
    "seriousnesslifethreatening",
    "seriousnesscongenitalanomali"
  ]

clean_dae_df = expanded_dae_df[keep_columns]

clean_dae_df = clean_dae_df.rename(columns = {
    "openfda.application_number": "application_number",
    "serious": "serious",
    "seriousnessdeath": "death",
    "seriousnessdisabling": "disabling",
    "seriousnessother": "other",
    "seriousnesshospitalization": "hospitalized",
    "seriousnesslifethreatening": "life_threatening",
    "seriousnesscongenitalanomali": "congenital_anomali"
})

clean_dae_df = clean_dae_df.explode("application_number")

clean_dae_df = clean_dae_df.dropna(subset = ["application_number"])

clean_dae_df['application_number'] = clean_dae_df['application_number'].str.lower()

clean_dae_df.to_csv("clean_dae.csv", index = False)

In [10]:
df = pd.read_csv("clean_dae.csv")
df

,application_number,serious,death,disabling,other,hospitalized,life_threatening,congenital_anomali
0,anda078682,1,NaN,NaN,1.0,NaN,NaN,NaN
1,anda077349,1,NaN,NaN,1.0,NaN,NaN,NaN
2,505g(a)(3),1,NaN,NaN,1.0,NaN,NaN,NaN
3,anda079174,1,NaN,NaN,1.0,NaN,NaN,NaN
4,anda210602,1,NaN,NaN,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
324155,anda040571,1,1.0,NaN,NaN,NaN,NaN,NaN
324156,anda215673,1,1.0,NaN,NaN,NaN,NaN,NaN
324157,anda040323,1,1.0,NaN,NaN,NaN,NaN,NaN
324158,anda218083,1,1.0,NaN,NaN,NaN,NaN,NaN


# Merge Datasets

**Joins the two datasets based on matching application numbers.**

In [11]:
clean_dpl_df = pd.read_csv("clean_dpl.csv", low_memory=False)
clean_dae_df = pd.read_csv("clean_dae.csv", low_memory=False)

merged_df = clean_dpl_df.merge(
    clean_dae_df,
    on="application_number",
    how="inner"
)

merged_df.to_csv("dpl_dae.csv", index=False)

In [12]:
df = pd.read_csv("dpl_dae.csv")
df

,active_ingredient,application_number,serious,death,disabling,other,hospitalized,life_threatening,congenital_anomali
0,povidoneiodine,m003,2,NaN,NaN,NaN,NaN,NaN,NaN
1,povidoneiodine,m003,1,NaN,NaN,NaN,1.0,NaN,NaN
2,povidoneiodine,m003,1,NaN,NaN,NaN,1.0,NaN,NaN
3,povidoneiodine,m003,1,NaN,NaN,NaN,1.0,NaN,NaN
4,povidoneiodine,m003,1,NaN,NaN,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2734426,polyethylene glycol,anda202850,2,NaN,NaN,NaN,NaN,NaN,NaN
2734427,polyethylene glycol,anda202850,1,NaN,NaN,NaN,1.0,NaN,NaN
2734428,polyethylene glycol,anda202850,1,NaN,NaN,1.0,1.0,1.0,NaN
2734429,polyethylene glycol,anda202850,1,NaN,NaN,1.0,1.0,NaN,NaN
